In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
# import openai
# from sentence_transformers import SentenceTransformer, util
import json
import csv
import re

test = pd.read_csv('MA.csv')
states = ['Alabama', 'Arkansas', 'Kentucky', 'Louisiana', 'Mississippi', 'Tennessee', 'Texas', 'Oklahoma','California', 'Oregon', 'Washington', 'Alaska', 'Hawaii','Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming','Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin','Pennsylvania', 'Vermont', 'West Virginia','District Of Columbia','Connecticut', 'Delaware', 'Florida', 'Georgia', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'North Carolina', 'Rhode Island', 'South Carolina', 'Virginia']
files = ['east_coast_1.csv','east_coast_2.csv','mountain_west.csv','south.csv','pacific_west.csv','midwest.csv']

FileNotFoundError: [Errno 2] No such file or directory: 'MA.csv'

## Testing block. Here we test different code and add them to different functios

In [32]:
pattern  = re.compile(r'\bfast food|fast-food|fastfood|chain|express\b', re.I)
ind = test[test['subtypes'].str.contains(pattern, na=False)].index
new_test = test.drop(ind)
ind_1 = new_test[new_test['description'].str.contains(pattern, na=False)].index
new_test = new_test.drop(ind_1)
print(test.shape[0])
print(new_test.shape[0])
new_test.to_csv('MA_testing.csv',index=False,encoding='utf-8')
# print(ind[ind[:]==True].index)
# print(MA)

1911
1811


## Adding information to sampled 4000 datasets

In [6]:
# this function grabs additional information we need for the filtered sample from the original national dataset
# info should be a list of categories from the national dataset we need.
def get_info(sample, original):
    new = sample.merge(original, how='left',on='national_id') #[['sample_id','national_id','name','location_link','site','us_state','type','subtypes','area','rating','has_name','CH_name','source','range','postal_code']]
    
#     sample.merge(original, left_on='national_id', right_on='national_id')[['sample_id','national_id','name','location_link','site','us_state','type','subtypes','area','rating','has_name','CH_name','source','range','postal_code']]
    print(sample.describe())
    new.to_csv('merged_sample.csv', index = False, encoding='utf-8')
    
sample = pd.read_csv(r'sample_original.csv')
national = pd.read_csv(r'national.csv')
get_info(sample, national)
    

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_2682/4159678023.py:11: DtypeWarning: Columns (35,39,43,59) have mixed types. Specify dtype option on import or set low_memory=False.
  national = pd.read_csv(r'national.csv')


         sample_id   national_id         area       rating     has_name
count  4018.000000   4018.000000  4018.000000  4018.000000  4018.000000
mean   2015.621205  16649.595819     1.680687     4.039970     0.486809
std    1164.164745   9583.567299     0.634130     0.428759     0.499888
min       0.000000      0.000000     0.000000     1.000000     0.000000
25%    1007.250000   8094.000000     2.000000     3.800000     0.000000
50%    2017.500000  16471.500000     2.000000     4.100000     0.000000
75%    3023.750000  24548.250000     2.000000     4.300000     1.000000
max    4029.000000  33275.000000     3.000000     5.000000     1.000000


### Separate Chinese restaurants from other businesses

###### We remove wrong queries from each region. We scraped our data by region, and each region only contains certain states, but sometimes restaurants from some other state appear in a regional dataset, so we need to remove them

###### We then removed the businesses that are actually not Chinese restaurants from the datasets

In [3]:
# this function focuses on a single column and returns the frequencies of the values in a csv
def get_freq(file, column,save_to):
    
    counts = {}
    for value in file[column]:
        counts[value] = counts.get(value, 0) + 1
    with open(save_to,'w') as f:
        w = csv.writer(f)
        w.writerows(counts.items())
    # types.to_csv(path+'types.csv',index=False,encoding='utf-8')

# this function returns the total number of items when given a list of datasets
def get_total(path,restaurants):
    total = 0
    for res in restaurants:
        df = pd.read_csv(path+res)
        total += df.shape[0]
    return total

#this function assembles the given datasets into one dataset
def assemble(path,files,savename):
    datasets=[]
    for file in files:
        df = pd.read_csv(path+file)
        datasets.append(df)
    df_total = pd.concat(datasets)
    # print(df_total['type'].value_counts())
    df_total.to_csv(savename,index=False,encoding='utf-8')
    # picking out unique value's frequencies in a column
    # counts = {}
    # for value in df_total['type']:
    #     counts[value] = counts.get(value, 0) + 1
    # with open(path+'types.csv','w') as f:
    #     w = csv.writer(f)
    #     w.writerows(counts.items())
    # types.to_csv(path+'types.csv',index=False,encoding='utf-8')
    # df_total.to_csv(path+'national.csv',index=False,encoding='utf-8')

assemble('datasets_chinese_filtered/',files,'national_chinese_res_filtered.csv')
# this function filters out businesses from states that don't belogn in a region. eg. MA in west coast
def filter_region(csvfile, states):
    df = pd.read_csv('raw_datasets/'+ csvfile)
    regions = df[df['us_state'].isin(states)]
    regions = regions.reset_index(drop = True)
    regions.to_csv('raw_datasets/'+csvfile, encoding='utf-8')
#     return df


east_coast_1 = ['Connecticut', 'Delaware', 'Florida', 'Georgia', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'North Carolina', 'Rhode Island', 'South Carolina', 'Virginia']
east_coast_2 = ['Pennsylvania', 'Vermont', 'West Virginia','District Of Columbia']
midwest = ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin']
mountain_west = ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming']
pacific_west = ['California', 'Oregon', 'Washington', 'Alaska', 'Hawaii']
south = ['Alabama', 'Arkansas', 'Kentucky', 'Louisiana', 'Mississippi', 'Tennessee', 'Texas', 'Oklahoma']

states = [east_coast_1, east_coast_2, mountain_west, south, pacific_west, midwest]

#execution code
# for i in range(6):
#     filter_region(files[i],states[i])
    
# print(get_total('raw_datasets/',files))
# print(get_total('datasets_state_cleaned/',files))

# this function filters out businesses that are not really chinese restaurants and put the filtered df in separate csv files
def filter_business(csvfile):
    filepath = 'datasets_state_cleaned/'+csvfile
    df = pd.read_csv(filepath)
    categories = ['Chinese restaurant', 'Sichuan restaurant', 'Shanghainese restaurant', 'Taiwanese restaurant', 'Hunan restaurant', 'Mandarin restaurant', 'Cantonese restaurant', 'Chinese takeaway', 'Chinese noodle restaurant','Dim sum restaurant','Dumpling restaurant','Delivery Chinese restaurant','Hot pot restaurant']
    cn_res = df[df['type'].isin(categories)]
    cn_res = cn_res.reset_index(drop = True)
#     non_res = df[~ df['type'].isin(categories)]
#     non_res = non_res.reset_index(drop = True)
#     print(cn_res)
    cn_res.to_csv('datasets_chinese_filtered/'+csvfile, index=False,encoding='utf-8')
#     non_res.to_csv('filtered_datasets/other_businesses/other_filtered_'+csvfile, encoding='utf-8')

#execution code    
# for file in files:
#     filter_business(file)
    
# print(get_total('datasets_chinese_filtered/',files))
# assemble('datasets_state_cleaned/',files)


### Drop Duplicates and fast food restaurants

In [39]:
# drops duplicate (chain) restaurants
# def drop(file):
#     # read csv
#     df = pd.read_csv('filtered_datasets/restaurants/filtered_'+file)
#     new_df = df.drop_duplicates(subset=['name'],keep = False)
#     new_df.to_csv('filtered_datasets/classified_res/'+file, encoding='utf-8')
# #     return new_df
def del_ff(path_from, path_to):
    df = pd.read_csv(path_from)
    # index_ff = df[df['subtypes'].str.match('fast food',case=False)|df['subtypes'].str.match('express',case=False)|df['description'].str.match('express',case=False)|df['description'].str.match('fast food',case=False)].index
    # df = df.drop(index_ff)
    # df = df.reset_index(drop = True)
    # print(df.shape[0])
    print(df.shape[0])
    pattern  = re.compile(r'\bfast food|fast-food|fastfood|chain|express\b', re.I)
    name_pattern = re.compile(r'\bfast food|fast-food|fastfood\b', re.I)
    ind = df[df['subtypes'].str.contains(pattern, na=False)|df['description'].str.contains(pattern, na=False)|df['reviews_tags'].str.contains(pattern, na=False)|df['name'].str.contains(name_pattern, na=False)].index
    df_final = df.drop(ind)
    # ind_1 = new_df[new_df['description'].str.contains(pattern, na=False)].index
    # df_final = new_df.drop(ind_1)
    print(df_final.shape[0])
    df_final.to_csv(path_to, index=False, encoding='utf-8')

# del_ff('national_chinese_res_filtered.csv','national_no_ff.csv')

# drops additional chain restaurants that have not been filtered due to naming
def del_chain(path_from, path_to):
#     chains = ['Manchu Wok','Asian Chao','City Wok','Chowking','Pick Up Stix','HuHot Mongolian Grill','Panda Express',"Mama Fu's","BD's Mongolian Grill",'Leeann Chin',"Mark Pi's",'Flat Top Grill','Big Bowl','Din Tai Fung',"P.F. Chang's",'Lao Sze Chuan','Pei Wei Asian Kitchen','Mr. Chow','Chinese Gourmet Express']
#     df = pd.read_csv('filtered_datasets/restaurants/filtered_'+file)
    df = pd.read_csv(path_from)
    #  delete chains
    pattern  = re.compile(r'\bB2J Fish Soup|Mama Fu\'s|Manchu Wok|Happy Lamb|BD\'s Mongolian Grill|panda express|Mao\'s Bao|P.F. Chang\'s|Leeann Chin|Xi\'an Famous Foods|Asian Chao|Mark Pi\'s|Ten Seconds Yunnan Rice Noodle|Flat Top Grill|Liuyishou|Big Bowl|A&J Restaurant|Haidilao|Boiling Point|Szechuan Impression|Din Tai Fung|City Wok|Bo Ling\'s|Lao Sze Chuan|Meizhou Dongpo|Chowking|Pei Wei|Yang\'s Braised Chicken Rice|Mr. Chow|Tasty Pot|Pick Up Stix|Dumplings of Fury|Chinese Gourmet Express|CHIKO|HuHot Mongolian Grill|Dagu Rice Noodle|ChiliSpot\b', re.I)
    # indexChain = re.compile(r'\bpanda express\b', re.I)
    # indexChain = df[(df['name'].str.contains("PANDA EXPRESS")) | (df['name'].str.contains("Panda express")) | (df['name'].str.contains("Mao's Bao")) | (df['name'].str.contains("P.F. Chang's")) | (df['name'].str.contains("Xi'an Famous Foods")) | (df['name'].str.contains('Ten Seconds Yunnan Rice Noodle')) | (df['name'].str.contains('Liuyishou')) | (df['name'].str.contains('Haidilao')) | (df['name'].str.contains('Boiling Point')) | (df['name'].str.contains('Szechuan Impression')) | (df['name'].str.contains("Bo Ling's")) | (df['name'].str.contains('Meizhou Dongpo')) | (df['name'].str.contains("Yang's Braised Chicken Rice")) | (df['name'].str.contains('Tasty Pot')) | (df['name'].str.contains('Dumplings of Fury')) | (df['name'].str.contains('CHIKO')) | (df['name'].str.contains('Dagu Rice Noodle')) | (df['name'].str.contains('ChiliSpot')) | (df['name'].str.contains('Happy Lamb')) | (df['name'].str.contains('Manchu Wok')) | (df['name'].str.contains('B2J Fish Soup 不二家酸菜鱼')) | (df['name'].str.contains('Asian Chao')) | (df['name'].str.contains('A&J Restaurant')) | (df['name'].str.contains('City Wok')) |(df['name'].str.contains('Chowking'))| (df['name'].str.contains('Pick Up Stix'))| (df['name'].str.contains('HuHot Mongolian Grill')) | (df['name'].str.contains('Panda Express')) | (df['name'].str.contains("Mama Fu's")) | (df['name'].str.contains("BD's Mongolian Grill")) | (df['name'].str.contains('Leeann Chin')) | (df['name'].str.contains("Mark Pi's")) | (df['name'].str.contains('Flat Top Grill')) | (df['name'].str.contains('Big Bowl')) | (df['name'].str.contains('Din Tai Fung')) | (df['name'].str.contains('Lao Sze Chuan')) | (df['name'].str.contains('Pei Wei')) | (df['name'].str.contains('Mr. Chow')) | (df['name'].str.contains('Chinese Gourmet Express'))].index
    indexChain = df[df['name'].str.contains(pattern, na=False)].index
    print(df.shape[0])
    df = df.drop(indexChain) #, inplace = True
#     print(df_filtered.describe())
    # deletes fast food
    # index_ff = df[df['subtypes'].str.contains('fast food')].index
    # indef_FF = df[df['subtypes'].str.contains('Fast food')].index
    
    # des_chain = df[df['description'].str.match('chain',case=False)].index
    # df = df.drop(des_chain)
    df = df.reset_index(drop = True)
    print(df.shape[0])
    df.to_csv(path_to, index=False, encoding='utf-8')
del_chain('national_no_ff.csv','national_no_hidden_chain.csv')
# files = ['east_coast_1.csv','east_coast_2.csv','east_coast_3.csv','mountain_west.csv','south.csv','west_coast.csv','midwest_1.csv','midwest_2.csv']
# for file in files:
#     drop(file)
# for file in files:
#     del_chain(file)
# for file in files:
#     drop_ff(file)

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_1660/2428536484.py:30: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_from)


31727
31548


## Recleaning the 4000 sample

### Classifying areas

In [ ]:
# this function returns an area's classification based on its RUCA2 score
ruca_df = pd.read_csv(r'RUCA.csv')

def get_area(score):
    if score in [1,1.1]:   # urban
        return 2
    elif score in [2, 2.1, 4, 4.1]:  # suburban
        return 1
    elif score == 999:
        return 3
    return 0 # rural


# returns a RUCA score based on the input postal code
def get_ruca(post):
    try:
        row_ind = ruca_df[ruca_df['ZIP_CODE'] == post].index.values    # is the row index here the real row index or the column row index?
        ruca_score = ruca_df.loc[row_ind, 'RUCA2']
        return ruca_score.iloc[0]
    except:
        print('zip code '+str(post)+' not found. Classified as other. Area number = 3')
        return 999

# returns the number classification of an area based on its postal code
def classify(post):
#     post = float(post)
    return get_area(get_ruca(post))

# this classifies each restaurant as urban/rural/suburban
# it takes in a file as an input and adds a colum "Area" to that file
def area_classify(file,new_name):
    # read csv
    df = pd.read_csv(file)
    areas = df['postal_code'].apply(classify)  # apply function on column postal_code
    df['Area'] = areas # add new column to dataframe
    df.to_csv(new_name,index=False, encoding='utf-8')
    return df

# files = ['east_coast_1.csv','east_coast_2.csv','east_coast_3.csv','mountain_west.csv','south.csv','west_coast.csv','midwest_1.csv','midwest_2.csv']
# for file in files:
# file=pd.read_csv('national_no_hidden_chain.csv')
area_classify('national_no_hidden_chain.csv','national_region_classified.csv')

### Filtering out datapoints in the buggy sample that exists in the new national dataset

In [9]:
nn_df = pd.read_csv('national_region_classified.csv')
bug_df = pd.read_csv('sample_full_info.csv')
clean_df = bug_df.google_id.isin(nn_df.google_id)
bug_df['in_national'] = clean_df
bug_df = bug_df[bug_df.in_national == True]
bug_df.to_csv('sample_cleaned.csv',index=False, encoding='utf-8')

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_2580/4006033773.py:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  nn_df = pd.read_csv('national_region_classified.csv')


### Comparing the state numbers between old and new sample

In [4]:
clean_df = pd.read_csv('sample_cleaned.csv')
old_df = pd.read_csv('sample_full_info.csv')
nn_df = pd.read_csv('national_region_classified.csv')
# print(clean_df['us_state'].value_counts())
states_new = clean_df['us_state'].value_counts()
states_old = old_df['us_state'].value_counts()
# print(states_new)
# print(states_old)
# get_freq(clean_df,'us_state','clean_states.csv')
# get_freq(old_df,'us_state','old_states.csv')

get_freq(nn_df,'us_state','all_states.csv')

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_1304/3083790615.py:3: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  nn_df = pd.read_csv('national_region_classified.csv')


### Calculating the percentage of state sample frequencies to national fequencies

In [39]:
sample_state = pd.read_csv('clean_states.csv') 
national_state = pd.read_csv('all_states.csv') 
print(sample_state.head())
print(national_state.head())

### code for merging
new = sample_state.merge(national_state, how='left',on='state') #[['sample_id','national_id','name','location_link','site','us_state','type','subtypes','area','rating','has_name','CH_name','source','range','postal_code']]
new.to_csv('merged_state_proportion.csv', index = False, encoding='utf-8')


         state   percentage
0  Connecticut           52
1     Delaware           47
2      Florida          157
3      Georgia           87
4        Maine           46
           state   percentage
0       Virginia          907
1       New York         2979
2        Georgia          899
3       Delaware          138
4  New Hampshire          177


### Expired: Compile National df with filtered Chinese restaurants

In [21]:
def national_df(file):
    df = pd.read_csv('datasets_chinese_filtered/'+file)
#     df = pd.read_csv('filtered_datasets/restaurants/filtered_'+file) 
    return df
    
def get_national():
    files = ['east_coast_1.csv','east_coast_2.csv','mountain_west.csv','south.csv','pacific_west.csv','midwest.csv']
    dfs = []
    for file in files:
        df = national_df(file) #.dropna(subset=['cid', 'reviews_id'])
        # print(df['cid'].dtype)
        df['cid'] = df['cid'].astype('string')
        df['reviews_id'] = df['reviews_id'].astype('string',errors='ignore')
        # df['reviews_id'] = df['reviews_id'].astype('int64',errors='ignore')
        dfs.append(df)
    national = pd.concat(dfs).reset_index(drop = True)
    print(national.shape[0])
    # national['reviews_id'].astype(int,errors='ignore')
#     national = national.drop_duplicates(subset=['name'],keep = False)
#     national[national.duplicated(subset=['name'],keep=False)].reset_index(drop = True).to_csv('filtered_datasets/classified_res/duplicates.csv', encoding='utf-8')
    national.to_csv('output/national_testing.csv', encoding='utf-8')
#     return national

get_national()

36424


### Making state dfs

In [5]:
national =  pd.read_csv(r'filtered_datasets/classified_res/national_nochains.csv')
def state_df(state):
    df = national[national['us_state'].isin([state])]
    df.to_csv('filtered_datasets/states/'+state+'.csv', encoding='utf-8')
def get_states():
    for state in states:
        state_df(state)
get_states()

### Count # restaurants in each dataframe

In [2]:
def df_count(file):
    df = pd.read_csv('filtered_datasets/classified_res/'+file)
    return df.count()[0]
total = 0
for file in files:
    total += df_count(file)
print(total)
# print(df_count('east_coast_1.csv'))

19788


### Categories of non-Chinese-restaurant businesses

In [ ]:

def get_categories(filename):
    df = pd.read_csv('filtered_datasets/other_businesses/other_filtered_'+ filename)
#     types = df['type'].unique()
    n = 10
    print(filename)
    print(df['type'].value_counts()[:n].index.tolist())
#     print(df['type'].describe())
    print("--------------------------------------------------------------------")
files = ['east_coast_1.csv','east_coast_2.csv','east_coast_3.csv','mountain_west.csv','south.csv','west_coast.csv','midwest_1.csv','midwest_2.csv']
# for file in files:
#     get_categories(file)
# get_categories('east_coast_1.csv')